In [30]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = "/content/drive/MyDrive/DSE4211/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import os
print(os.listdir("/content/drive/MyDrive/DSE4211/"))


['btc_price_data.csv', 'apple_price_data.csv', 'JNJ_price_data.csv', 'Xiaomi_price_data.csv', 'BNB_price_data.csv', 'USDT_price_data.csv', 'Gold(Tether)_price_data.csv', 'Xiaomi_price_data.gsheet']


In [33]:
# Import necessary libraries
import numpy as np
import pandas as pd
import scipy.optimize as sco
from google.colab import drive


# Define file paths
file_paths = {
    "Apple": "/content/drive/MyDrive/DSE4211/apple_price_data.csv",
    "BNB": "/content/drive/MyDrive/DSE4211/BNB_price_data.csv",
    "Bitcoin": "/content/drive/MyDrive/DSE4211/btc_price_data.csv",
    "JNJ": "/content/drive/MyDrive/DSE4211/JNJ_price_data.csv",
    "USDT": "/content/drive/MyDrive/DSE4211/USDT_price_data.csv",
    "Xiaomi": "/content/drive/MyDrive/DSE4211/Xiaomi_price_data.csv",
}

# Load and preprocess data
dfs = {}
for asset, path in file_paths.items():
    df = pd.read_csv(path)
    df.index = pd.date_range(start="2023-01-01", periods=len(df), freq="D")
    if "Close" in df.columns:
        df = df[["Close"]].rename(columns={"Close": asset})
    dfs[asset] = df

merged_df = pd.concat(dfs.values(), axis=1)  # Merge all assets
merged_df = merged_df.iloc[2:].apply(pd.to_numeric, errors='coerce')
merged_df.fillna(method='ffill', inplace=True)  # Forward-fill missing values

print("Cleaned Data (First 5 Rows):")
print(merged_df.head())

# Compute daily returns
returns = merged_df.pct_change().dropna()

# Exclude USDT from optimization
returns_ex_usdt = returns.drop(columns=["USDT"], errors="ignore")

# Compute expected return and covariance matrix
expected_returns = returns_ex_usdt.mean().values  # Mean returns
cov_matrix = returns_ex_usdt.cov().values        # Risk (covariance)
num_assets = len(returns_ex_usdt.columns)

import yfinance as yf

# Fetch US 3-month T-bill yield
rf_tbill = yf.Ticker("^IRX")
rf_annual = rf_tbill.history(period="1d")["Close"].iloc[-1] / 100  # Convert to decimal
risk_free_rate_us = rf_annual / 252  # Convert to daily rate


# Fetch HK 3-month Treasury yield
rf_annual_hk = 3.364 / 100 #2025/03/18
risk_free_rate_hk = rf_annual_hk / 252


rf_annual_btc = 0.07  # 7% annual rate (NEXO)
risk_free_rate_btc = rf_annual_btc / 252  # Convert to daily rate (252 trading days in a year)

rf_annual_bnb = 0.12  # 12% annual rate for BNB (NEXO)
risk_free_rate_bnb = rf_annual_bnb / 252  # Convert to daily rate (252 trading days in a year)

risk_free_rates = {
    "Apple": risk_free_rate_us,       # US (3-month T-bill)
    "BNB": risk_free_rate_bnb,   # BNB DeFi lending rate
    "Bitcoin": risk_free_rate_btc,  # BTC DeFi lending rate
    "JNJ": risk_free_rate_us,        # US (3-month T-bill)
    "Gold_Tether": risk_free_rate_us,   # Gold tether, US-based
    "Xiaomi": risk_free_rate_hk,    # Xiaomi (HK 3-month T-bill)
}

# Print risk-free rates per asset
for asset, rf in risk_free_rates.items():
    print(f"Daily Risk-Free Rate ({asset}): {rf:.6f}")


# Define Sharpe Ratio objective function (maximize return/risk)
def sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rates):
    excess_returns = expected_returns - np.array([risk_free_rates[asset] for asset in returns_ex_usdt.columns])
    port_return = np.dot(weights, excess_returns)  # Portfolio excess return
    port_volatility = np.sqrt(weights.T @ cov_matrix @ weights)  # Portfolio volatility
    return - port_return / port_volatility  # Negative Sharpe Ratio


# Initial equal weight allocation
init_weights = np.ones(num_assets) / num_assets


# Constraint: weights must sum to 95% (USDT is fixed at 5%)
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 0.95})

#bounds = tuple((0, 1) for _ in range(num_assets))

# Optimize to maximize Sharpe Ratio (minimize negative Sharpe)
opt_result = sco.minimize(sharpe_ratio, init_weights, args=(expected_returns, cov_matrix, risk_free_rates),
                          method='SLSQP',  constraints=constraints)

# Extract optimized weights
allocation_weights = opt_result.x

# Create allocation DataFrame
allocation_df = pd.DataFrame({"Asset": returns_ex_usdt.columns, "Optimal Allocation (%)": allocation_weights * 100})

usdt_allocation = pd.DataFrame({"Asset": ["USDT"], "Optimal Allocation (%)": [5.0]})
allocation_df = pd.concat([allocation_df, usdt_allocation], ignore_index=True)

print("\nOptimal Portfolio Allocation (Mean-Variance Optimization):")
print(allocation_df)


Cleaned Data (First 5 Rows):
                 Apple         BNB       Bitcoin         JNJ      USDT  Xiaomi
2023-01-03  123.632515  244.136978  16625.080078  166.303696  0.999692   11.22
2023-01-04  124.907715  245.535904  16688.470703  168.114288  0.999771   11.50
2023-01-05  123.583092  246.133362  16679.857422  166.873016  0.999759   11.62
2023-01-06  128.130234  259.119690  16863.238281  168.226303  0.999763   11.24
2023-01-07  128.654144  256.422852  16836.736328  163.867798  0.999697   12.10
Daily Risk-Free Rate (Apple): 0.000166
Daily Risk-Free Rate (BNB): 0.000476
Daily Risk-Free Rate (Bitcoin): 0.000278
Daily Risk-Free Rate (JNJ): 0.000166
Daily Risk-Free Rate (Gold_Tether): 0.000166
Daily Risk-Free Rate (Xiaomi): 0.000133

Optimal Portfolio Allocation (Mean-Variance Optimization):
     Asset  Optimal Allocation (%)
0    Apple               70.335815
1      BNB               -4.025328
2  Bitcoin               37.983408
3      JNJ              -44.314539
4   Xiaomi             

<ipython-input-33-1d68b1adb60a>:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df.fillna(method='ffill', inplace=True)  # Forward-fill missing values
